In [ ]:
# necessay requirements
!pip install huggingface_hub langchain langchain_community chromadb sentence-transformers


In [ ]:
!pip install langchain-huggingface

In [ ]:
!pip install llama_index==0.10.17
!pip install -q llama-index-llms-gemini

  Using cached llama_index-0.10.17-py3-none-any.whl (5.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.


In [ ]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = "Use your own"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from langchain.document_loaders import TextLoader

docs_folder = '/content/drive/My Drive/llmctx'
documents = []

for filename in os.listdir(docs_folder):
    if filename.endswith('.txt'):
        filepath = os.path.join(docs_folder, filename)
        loader = TextLoader(filepath)
        documents.extend(loader.load())

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(documents, embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def get_context_from_question(question, vectordb, k=3):
    docs = vectordb.similarity_search(question, k=k)
    context = " ".join([doc.page_content for doc in docs])
    return context

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Given the following context, please provide a detailed and accurate answer to the question.

Context: {context}

Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=HuggingFaceHub(repo_id="google/flan-t5-small", model_kwargs={"temperature": 20, "max_length": 256})
)

# llm_chain = LLMChain(
#     prompt=prompt,
#     llm=HuggingFaceHub(repo_id="deepset/roberta-base-squad2", model_kwargs={"temperature": 0, "max_length": 128}, task="conversational")
# )

# llm_chain = LLMChain(
#     prompt=prompt,
#     llm=HuggingFaceHub(repo_id="deepset/minilm-uncased-squad2", model_kwargs={"temperature": 0, "max_length": 128}, task="conversational")
# )



In [ ]:
question = "3 best movies according to wikipedia"
context = get_context_from_question(question, vectordb)
#print(context)
result = llm_chain.run({"context": context, "question": question})
print(result)


KeyboardInterrupt: 